In [29]:
import numpy as np
import copy
import matplotlib.pyplot as plt
import sklearn
import sklearn.datasets
import sklearn.linear_model
from PIL import Image
import os
from tqdm import tqdm
from sklearn.utils import shuffle 
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import tensorflow as tf

import warnings
warnings.filterwarnings("ignore")

In [15]:
import tensorflow as tf
print(tf.__version__)


2.18.0


In [30]:
# Verimizde iki tür sınıf vardır kedi ve köpek
labels = ["dogs","cats"]

X_train = []
y_train = []

X_test = []
y_test = []

image_size = 28

# Bu adımda görüntü veri kümesini eğitim (training_set) ve test (test_set) olarak ikiye ayırarak yükler.
# Veriler, data/ klasörünün içindeki training_set/ ve test_set/ dizinlerinden okunur.
for label in labels:
    folderPath = os.path.join("..","data","training_set", label)
    for file in tqdm(os.listdir(folderPath)):
        
        img = Image.open(os.path.join(folderPath, file))
        img = img.resize((image_size, image_size))
      
        X_train.append(img)
        y_train.append(label)
        
for label in labels:
    folderPath = os.path.join("..","data","test_set", label)
    for file in tqdm(os.listdir(folderPath)):
        
        img = Image.open(os.path.join(folderPath, file))
        img = img.resize((image_size, image_size))
      
        X_test.append(img)
        y_test.append(label)
     

100%|██████████| 1011/1011 [00:02<00:00, 372.36it/s]


In [17]:
print(y_test)

['dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs',

ValueError: np.int64(1) is not in list

In [19]:
y_test.shape

(1, 4046)

In [20]:
def sigmoid(z):
    s = 1+(1+np.exp(-z))
    return s

In [21]:
def katman_boyutlari(X, Y):
    # n_x verideki bir örneğin sahip olduğu özellik sayısı
    # n_h gizli katman boyutu
    # n_y sınıflandırılacak sınıf sayısı

    n_x = X.shape[0]
    n_h = 4
    n_y = Y.shape[0]

    return (n_x, n_h, n_y)

In [ ]:
def paramtetre_başlat(n_x, n_h, n_y):

    # parametrelerimizi uygun boyutlarda başlatıyoruz
    W1 = np.random.randn(n_h, n_x)*0.01
    b1 = np.zeros((n_h,1))

    w2 = np.random.randn(n_y, n_h)*0.01
    b2 = np.zeros((n_y, 1))

    parameters = {"W1":W1,
                  "b1":b1,
                  "W2":w2,
                  "b2":b2}
    return parameters

In [23]:
def ileri_yayilim(X, parameters):
    # parametrelerimizi çekiyoruz
    W1 = parameters["W1"]
    b1 = parameters["b1"]
    W2 = parameters["W2"]
    b2 = parameters["b2"]

    Z1 = np.dot(W1,X)+b1
    # tanh fonksiyonu sigmoid fonksiyonundan genel durumlarda daha iyi performans gösterir
    # sigmoid fonksiyonun öne çıktığı tek durum ikili sınıflandırma yapılacağı zamandır 
    A1 = np.tanh(Z1)
    Z2 = np.dot(W2,A1)+b2
    A2 = sigmoid(Z2)

    cache = {"Z1":Z1,
             "A1":A1,
             "Z2":Z2,
             "A2":A2}
    
    return A2, cache

In [24]:
def maliyet_hesaplama(A2, Y):
    # Örnek sayısı
    m = Y.shape[1]

    """
    logprobs = np.multiply(np.log(A2),Y)+np.multiply((1-Y),(np.log(1-A2)))
    cost = -np.sum(logprobs)/m

    cost = float(np.squeeze(cost)) # (1,1) boyutundaki matrisi skaler bir sayıya çevirir

    return cost
    """
    logprobs = np.multiply(np.log(A2),Y)+np.multiply((1-Y),(np.log(1-A2)))
    cost = -np.sum(logprobs)/m
    
    # YOUR CODE ENDS HERE
    
    cost = float(np.squeeze(cost))  # makes sure cost is the dimension we expect. 
                                    # E.g., turns [[17]] into 17 
    
    return cost

In [25]:
def geri_yayilim(parameters, cache, X, Y):

    m = Y.shape[1]

    W1 = parameters["W1"]
    W2 = parameters["W2"]

    A1 = cache["A1"]
    A2 = cache["A2"]

    # parametrelerimizin türevlerini alıyoruz
    dZ2 = A2-Y
    dW2 = np.dot(dZ2,A1.T)/m
    db2 = np.sum(dZ2, axis=1, keepdims=True)
    dZ1 = np.dot(W2.T, dZ2)*(1-np.power(A1,2))
    dW1 = np.dot(dZ1,X.T)/m
    db1 = np.sum(dZ1, axis=1, keepdims=True)/m

    grads = {"dW1":dW1,
             "db1":db1,
             "dW2":dW2,
             "db2":db2}
    
    return grads

In [26]:
def parametre_güncelleme(parameters, grads, learning_rate=1.2):

    # parametrelerimizi çekiyoruz
    W1 = parameters["W1"]
    b1 = parameters["b1"]
    W2 = parameters["W2"]
    b2 = parameters["b2"]

    dW1 = grads["dW1"]
    db1 = grads["db1"]
    dW2 = grads["dW2"]
    db2 = grads["db2"]

    # verilen learning_rate değeriyle parametrelerimizi güncelliyoruz
    W1 = W1 - learning_rate*dW1
    b1 = b1 - learning_rate*db1
    W2 = W2 - learning_rate*dW2
    b2 = b2 - learning_rate*db2

    parameters = {"W1":W1,
                  "b1":b1,
                  "W2":W2,
                  "b2":b2}
    
    return parameters


In [27]:
def nn_model(X, Y, n_h, num_iterations=10000, print_cost=False):

    # oluşturduğumuz fonksiyonları tek bir model altında birleştirelim
    np.random.seed(3)
    n_x = katman_boyutlari(X, Y)[0]
    n_y = katman_boyutlari(X, Y)[2]

    parameters = paramtetre_başlat(n_x, n_h, n_y)

    # verilen num_iterations değeri kadar döngü işlemi uygulicaz
    for i in range(num_iterations):

        A2, cache = ileri_yayilim(X, parameters)
        cost = maliyet_hesaplama(A2, Y)
        grads = geri_yayilim(parameters, cache, X, Y)
        parameters = parametre_güncelleme(parameters, grads)

        if print_cost and i % 1000 == 0:
            print ("Cost after iteration %i: %f" %(i, cost))
    
    return parameters

In [28]:
nn_model(X_train, y_train, n_h=4, num_iterations=2000, print_cost=True)

ValueError: operands could not be broadcast together with shapes (1,8005) (1,16010) 